In [2]:
# basic imports
import pandas as pd
import numpy as np
import os

In [28]:
filename=os.path.join('data','CrowdstormingDataJuly1st.csv') 
df = pd.read_csv(filename)
df = df.dropna(subset=['rater1', 'rater2'])

In [29]:
df['black'] = df['rater1'] + df['rater2'] > 1

In [36]:
df.size

3614009

In [37]:
df2 = df[['playerShort', 'club']]

In [76]:
df3 = df.groupby('playerShort', as_index=False)

In [79]:
def first(x):
    return x.iloc[0]
df4 = df3.agg({'games': np.sum, 'victories': np.sum, 
         'club': first, 'leagueCountry': first,
         'birthday': first, 'height': first,
         'weight': first, 'position': first,
         'games': np.sum, 'victories': np.sum,
         'ties':np.sum, 'defeats': np.sum,
         'goals': np.sum, 'yellowCards': np.sum,
         'yellowReds': np.sum, 'redCards': np.sum,
         'meanIAT': np.mean, 'meanExp': np.mean,
         'black': first})

In [80]:
df4

,playerShort,weight,leagueCountry,defeats,club,birthday,height,meanIAT,yellowReds,victories,games,goals,ties,position,redCards,meanExp,yellowCards,black
0,aaron-hughes,71.0,England,228,Fulham FC,08.11.1979,182.0,0.346459,0,247,654,9,179,Center Back,0,0.494575,19,False
1,aaron-hunt,73.0,Germany,122,Werder Bremen,04.09.1986,183.0,0.348818,0,141,336,62,73,Attacking Midfielder,1,0.449220,42,False
2,aaron-lennon,63.0,England,115,Tottenham Hotspur,16.04.1987,165.0,0.345893,0,200,412,31,97,Right Midfielder,0,0.491482,11,False
3,aaron-ramsey,76.0,England,68,Arsenal FC,26.12.1990,178.0,0.346821,0,150,260,39,42,Center Midfielder,1,0.514693,31,False
4,abdelhamid-el-kaoutari,73.0,France,43,Montpellier HSC,17.03.1990,180.0,0.331600,4,41,124,1,40,Center Back,2,0.335587,8,False
5,abdou-traore_2,74.0,France,33,Girondins Bordeaux,17.01.1988,180.0,0.320079,1,41,97,3,23,Right Midfielder,0,0.296562,11,True
6,abdoulaye-diallo_2,80.0,France,8,Stade Rennes,30.03.1992,189.0,0.341625,0,8,24,0,8,Goalkeeper,0,0.400818,0,True
7,abdoulaye-keita_2,83.0,France,2,Girondins Bordeaux,19.08.1990,188.0,0.355406,0,0,3,0,1,Goalkeeper,0,0.417225,0,True
8,abdoulwhaid-sissoko,68.0,France,62,Stade Brest,20.03.1990,180.0,0.348178,0,34,121,3,25,Defensive Midfielder,2,0.429630,21,True
9,abdul-rahman-baba,70.0,Germany,25,SpVgg Greuther Fürth,02.07.1994,179.0,0.342072,0,17,50,0,8,Left Fullback,1,0.361068,3,True
